In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))


/var/folders/lj/2t8twtsx5v1dw1dkbr59j1xr0000gn/T/ipykernel_46744/3522731578.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [3]:
# More aesthetically pleasing version
# 4-team tournament: EV Deltas, Conditional EV Deltas, & Unconditioned Tournament EV (Semifinals & Championship)

import mysql.connector
from collections import defaultdict

##############################################################################
# 1) Database Query for Conference/Championship Payouts
##############################################################################

conn = mysql.connector.connect(
    host='betting-db.cp86ssaw6cm7.us-east-1.rds.amazonaws.com',
    user='admin',
    password='7nRB1i2&A-K>',
    database='betting_db'
)

cursor = conn.cursor()

query = """
    SELECT 
        legs.ParticipantName,
        legs.EventType,
        SUM(bets.PotentialPayout) AS total_payout
    FROM bets
    JOIN legs ON bets.WagerID = legs.WagerID
    WHERE 
        bets.LegCount = 1
        AND bets.WLCA = 'Active'
        AND bets.WhichBankroll = 'GreenAleph'
        AND legs.LeagueName = 'NBA'
        AND legs.EventType IN ('Conference Winner', 'Championship')
    GROUP BY legs.ParticipantName, legs.EventType;
"""

cursor.execute(query)

payouts = defaultdict(lambda: {'conference': 0.0, 'championship': 0.0})
for participant_name, event_type, total_payout in cursor.fetchall():
    normalized_name = participant_name.strip().lower()
    etype = event_type.strip().lower()
    if etype == 'conference winner':
        payouts[normalized_name]['conference'] = float(total_payout)
    elif etype == 'championship':
        payouts[normalized_name]['championship'] = float(total_payout)

cursor.close()
conn.close()

##############################################################################
# 2) Team Data: Semifinal & Championship Probabilities
##############################################################################

team_probabilities = {
    "minnesota timberwolves": {
        "semifinal_prob": 0.233,
        "championship_probs": {
            "new york knicks": 0.58,
            "indiana pacers":   0.602
        }
    },
    "oklahoma city thunder": {
        "semifinal_prob": 0.767,
        "championship_probs": {
            "new york knicks": 0.767,
            "indiana pacers":   0.798
        }
    },
    "new york knicks": {
        "semifinal_prob": 0.57,
        "championship_probs": {
            "minnesota timberwolves": 0.42,
            "oklahoma city thunder":      0.233
        }
    },
    "indiana pacers": {
        "semifinal_prob": 0.43,
        "championship_probs": {
            "minnesota timberwolves": 0.398,
            "oklahoma city thunder":      0.202
        }
    },
}

##############################################################################
# 3) Conferences (2 teams each)
##############################################################################

conference_a = ["minnesota timberwolves", "oklahoma city thunder"]
conference_b = ["new york knicks", "indiana pacers"]
all_teams = conference_a + conference_b

##############################################################################
# 4) Portfolio EV if 'winning_team' wins its semifinal
##############################################################################

def calculate_portfolio_ev(winning_team, other_conference_teams):
    conference_winner_ev = payouts[winning_team]['conference']
    for team in other_conference_teams:
        conference_winner_ev += (
            team_probabilities[team]['semifinal_prob'] * payouts[team]['conference']
        )

    championship_ev = 0.0
    for opponent, win_prob in team_probabilities[winning_team]["championship_probs"].items():
        opponent_prob = team_probabilities[opponent]["semifinal_prob"]
        championship_ev += (
            win_prob * opponent_prob * payouts[winning_team]["championship"]
          + (1 - win_prob) * opponent_prob * payouts[opponent]["championship"]
        )
    
    return conference_winner_ev + championship_ev

##############################################################################
# 5) Conditional Portfolio EV
##############################################################################

def calculate_conditional_portfolio_ev(winning_team, other_conference_teams, conditioned_team):
    original_probs = {
        tm: team_probabilities[tm]['semifinal_prob']
        for tm in other_conference_teams
    }

    for tm in other_conference_teams:
        team_probabilities[tm]['semifinal_prob'] = 1.0 if tm == conditioned_team else 0.0

    ev = calculate_portfolio_ev(winning_team, other_conference_teams)

    for tm, prob in original_probs.items():
        team_probabilities[tm]['semifinal_prob'] = prob

    return ev

##############################################################################
# 6) Compute EV Deltas & Conditional EVs
##############################################################################

ev_results = {}
conditional_ev_results = defaultdict(dict)

for winner in all_teams:
    other_conference = conference_b if winner in conference_a else conference_a
    ev_results[winner] = calculate_portfolio_ev(winner, other_conference)
    for conditioned_team in other_conference:
        conditional_ev_results[winner][conditioned_team] = (
            calculate_conditional_portfolio_ev(winner, other_conference, conditioned_team)
        )

ev_deltas = {
    "Western Conference Finals": abs(ev_results["minnesota timberwolves"] - ev_results["oklahoma city thunder"]),
    "Eastern Conference Finals": abs(ev_results["new york knicks"] - ev_results["indiana pacers"]),
}

##############################################################################
# 7) Unconditioned Tournament Portfolio EV
##############################################################################

def calculate_unconditioned_tournament_portfolio_ev():
    total_ev = 0.0
    for team in all_teams:
        p_conf_win = team_probabilities[team]["semifinal_prob"]
        opp_conference = conference_b if team in conference_a else conference_a

        sum_final_meet = sum(
            team_probabilities[opp]["semifinal_prob"] *
            team_probabilities[team]["championship_probs"][opp]
            for opp in opp_conference
        )
        p_team_champ = p_conf_win * sum_final_meet

        total_ev += (
            p_conf_win * payouts[team]["conference"]
          + p_team_champ * payouts[team]["championship"]
        )

    return total_ev

tournament_portfolio_ev = calculate_unconditioned_tournament_portfolio_ev()

##############################################################################
# 8) Print Results
##############################################################################

# A) Scenario-based EV
print("==== Portfolio EVs ====")
print(f"Portfolio EV if Minnesota Timberwolves wins semifinal: ${ev_results['minnesota timberwolves']:,.2f}")
print(f"Portfolio EV if Oklahoma City Thunder wins semifinal:  ${ev_results['oklahoma city thunder']:,.2f}")
print(f"EV Delta (Western Conference Finals): ${ev_deltas['Western Conference Finals']:,.2f}\n")

print(f"Portfolio EV if New York Knicks wins semifinal: ${ev_results['new york knicks']:,.2f}")
print(f"Portfolio EV if Indiana Pacers wins semifinal:  ${ev_results['indiana pacers']:,.2f}")
print(f"EV Delta (Eastern Conference Finals): ${ev_deltas['Eastern Conference Finals']:,.2f}\n")

# B) Conditional EVs
print("==== Conditional Portfolio EVs ====")
for conditioned_team in all_teams:
    affected_teams = conference_b if conditioned_team in conference_a else conference_a
    print(f"\n--- If {conditioned_team.title()} already won in the other bracket ---")
    for tm in affected_teams:
        val = conditional_ev_results[tm][conditioned_team]
        print(f"If {tm.title()} wins semifinal: ${val:,.2f}")

# C) Unconditioned Tournament EV
print("\n==== Unconditioned Tournament Portfolio EV ====")
print(f"Semifinals & Championship (fully unconditional): ${tournament_portfolio_ev:,.2f}")


==== Portfolio EVs ====
Portfolio EV if Minnesota Timberwolves wins semifinal: $606,667.73
Portfolio EV if Oklahoma City Thunder wins semifinal:  $188,574.32
EV Delta (Western Conference Finals): $418,093.41

Portfolio EV if New York Knicks wins semifinal: $210,811.96
Portfolio EV if Indiana Pacers wins semifinal:  $385,644.80
EV Delta (Eastern Conference Finals): $174,832.84

==== Conditional Portfolio EVs ====

--- If Minnesota Timberwolves already won in the other bracket ---
If New York Knicks wins semifinal: $480,232.55
If Indiana Pacers wins semifinal: $774,267.84

--- If Oklahoma City Thunder already won in the other bracket ---
If New York Knicks wins semifinal: $128,967.11
If Indiana Pacers wins semifinal: $267,588.52

--- If New York Knicks already won in the other bracket ---
If Minnesota Timberwolves wins semifinal: $480,232.55
If Oklahoma City Thunder wins semifinal: $128,967.11

--- If Indiana Pacers already won in the other bracket ---
If Minnesota Timberwolves wins semi

In [4]:
# 4-team tournament: EV Deltas, Conditional EV Deltas, & Unconditioned Tournament EV (Semifinals & Championship)

import mysql.connector
from collections import defaultdict

# Connect to the database
conn = mysql.connector.connect(
    host='betting-db.cp86ssaw6cm7.us-east-1.rds.amazonaws.com',
    user='admin',
    password='7nRB1i2&A-K>',
    database='betting_db'
)

cursor = conn.cursor()

# Query to get PotentialPayouts for conference and championship bets
query = """
    SELECT 
        legs.ParticipantName,
        legs.EventType,
        SUM(bets.PotentialPayout) AS total_payout
    FROM bets
    JOIN legs ON bets.WagerID = legs.WagerID
    WHERE 
        bets.LegCount = 1
        AND bets.WLCA = 'Active'
        AND bets.WhichBankroll = 'GreenAleph'
        AND legs.LeagueName = 'NFL'
        AND legs.EventType IN ('Conference Winner', 'Championship')
    GROUP BY legs.ParticipantName, legs.EventType;
"""

cursor.execute(query)

# Store payouts in a dictionary
payouts = defaultdict(lambda: {'conference': 0, 'championship': 0})
for participant_name, event_type, total_payout in cursor.fetchall():
    normalized_name = participant_name.strip().lower()
    if event_type.strip().lower() == 'conference winner':
        payouts[normalized_name]['conference'] = float(total_payout)
    elif event_type.strip().lower() == 'championship':
        payouts[normalized_name]['championship'] = float(total_payout)

cursor.close()
conn.close()

# Matchups and probabilities (example values for proof of concept)
team_probabilities = {
    "kansas city chiefs": {"semifinal_prob": 0.54, "championship_probs": {"washington commanders": 0.67, "philadelphia eagles": 0.51}},
    "buffalo bills": {"semifinal_prob": 0.46, "championship_probs": {"washington commanders": 0.68, "philadelphia eagles": 0.52}},
    "washington commanders": {"semifinal_prob": 0.28, "championship_probs": {"kansas city chiefs": 0.33, "buffalo bills": 0.32}},
    "philadelphia eagles": {"semifinal_prob": 0.72, "championship_probs": {"kansas city chiefs": 0.49, "buffalo bills": 0.48}},
}

# Function to calculate Portfolio EV
def calculate_portfolio_ev(winning_team, other_conference_teams):
    conference_winner_ev = payouts[winning_team]['conference']
    for team in other_conference_teams:
        conference_winner_ev += team_probabilities[team]['semifinal_prob'] * payouts[team]['conference']
    
    championship_ev = 0
    for opponent, win_prob in team_probabilities[winning_team]["championship_probs"].items():
        opponent_prob = team_probabilities[opponent]["semifinal_prob"]
        championship_ev += (
            win_prob * opponent_prob * payouts[winning_team]["championship"] +
            (1 - win_prob) * opponent_prob * payouts[opponent]["championship"]
        )
    
    return conference_winner_ev + championship_ev

# Function to calculate Tournament Portfolio EV
def calculate_tournament_portfolio_ev():
    total_ev = 0
    for team, data in team_probabilities.items():
        # Compute probability of the team winning the championship
        championship_prob = sum(
            win_prob * team_probabilities[opponent]["semifinal_prob"]
            for opponent, win_prob in data["championship_probs"].items()
        )
        championship_prob *= data["semifinal_prob"]
        
        # Calculate total portfolio EV contribution
        total_ev += (
            data["semifinal_prob"] * payouts[team]["conference"] +
            championship_prob * payouts[team]["championship"]
        )
    
    return total_ev

# Teams in the two conferences
conference_a = ["kansas city chiefs", "buffalo bills"]
conference_b = ["washington commanders", "philadelphia eagles"]

# Calculate portfolio EVs for each scenario
ev_results = {}
conditional_ev_results = defaultdict(dict)

for winner in conference_a + conference_b:
    other_conference = conference_b if winner in conference_a else conference_a
    ev_results[winner] = calculate_portfolio_ev(winner, other_conference)

    for conditioned_team in other_conference:
        conditional_ev_results[winner][conditioned_team] = calculate_conditional_portfolio_ev(winner, other_conference, conditioned_team)

# Compute EV deltas
ev_deltas = {
    "AFC Semifinal": abs(ev_results["kansas city chiefs"] - ev_results["buffalo bills"]),
    "NFC Semifinal": abs(ev_results["washington commanders"] - ev_results["philadelphia eagles"]),
}

# Compute Tournament Portfolio EV
tournament_portfolio_ev = calculate_tournament_portfolio_ev()

# Print results
print("==== Portfolio EVs ====")
print(f"Portfolio EV if Kansas City Chiefs wins semifinal: ${ev_results['kansas city chiefs']:,.2f}")
print(f"Portfolio EV if Buffalo Bills wins semifinal: ${ev_results['buffalo bills']:,.2f}")
print(f"EV Delta: ${ev_deltas['AFC Semifinal']:,.2f}\n")

print(f"Portfolio EV if Washington Commanders wins semifinal: ${ev_results['washington commanders']:,.2f}")
print(f"Portfolio EV if Philadelphia Eagles wins semifinal: ${ev_results['philadelphia eagles']:,.2f}")
print(f"EV Delta: ${ev_deltas['NFC Semifinal']:,.2f}\n")

print("==== Conditional Portfolio EVs ====")
for conditioned_team, teams in {
    "kansas city chiefs": conference_b,
    "buffalo bills": conference_b,
    "washington commanders": conference_a,
    "philadelphia eagles": conference_a
}.items():
    print(f"\n--- If {conditioned_team.title()} already won ---")
    for team in teams:
        print(f"If {team.title()} wins semifinal: ${conditional_ev_results[team][conditioned_team]:,.2f}")

print("\n==== Tournament Portfolio EV (Semifinals & Championship) ====")
print(f"Unconditioned Tournament Portfolio EV: ${tournament_portfolio_ev:,.2f}")


==== Portfolio EVs ====
Portfolio EV if Kansas City Chiefs wins semifinal: $244,026.39
Portfolio EV if Buffalo Bills wins semifinal: $47,758.46
EV Delta: $196,267.92

Portfolio EV if Washington Commanders wins semifinal: $127,342.75
Portfolio EV if Philadelphia Eagles wins semifinal: $164,009.96
EV Delta: $36,667.22

==== Conditional Portfolio EVs ====

--- If Kansas City Chiefs already won ---
If Washington Commanders wins semifinal: $235,819.90
If Philadelphia Eagles wins semifinal: $247,217.80

--- If Buffalo Bills already won ---
If Washington Commanders wins semifinal: $0.00
If Philadelphia Eagles wins semifinal: $66,331.20

--- If Washington Commanders already won ---
If Kansas City Chiefs wins semifinal: $235,819.90
If Buffalo Bills wins semifinal: $0.00

--- If Philadelphia Eagles already won ---
If Kansas City Chiefs wins semifinal: $247,217.80
If Buffalo Bills wins semifinal: $66,331.20

==== Tournament Portfolio EV (Semifinals & Championship) ====
Unconditioned Tournament Po

In [3]:
# missing unconditioned tournament EV
# 4-team tournament: EV Deltas and Conditional EV Deltas

import mysql.connector
from collections import defaultdict

# Connect to the database
conn = mysql.connector.connect(
    host='betting-db.cp86ssaw6cm7.us-east-1.rds.amazonaws.com',
    user='admin',
    password='7nRB1i2&A-K>',
    database='betting_db'
)

cursor = conn.cursor()

# Query to get PotentialPayouts for conference and championship bets
query = """
    SELECT 
        legs.ParticipantName,
        legs.EventType,
        SUM(bets.PotentialPayout) AS total_payout
    FROM bets
    JOIN legs ON bets.WagerID = legs.WagerID
    WHERE 
        bets.LegCount = 1
        AND bets.WLCA = 'Active'
        AND bets.WhichBankroll = 'GreenAleph'
        AND legs.LeagueName = 'NFL'
        AND legs.EventType IN ('Conference Winner', 'Championship')
    GROUP BY legs.ParticipantName, legs.EventType;
"""

cursor.execute(query)

# Store payouts in a dictionary
payouts = defaultdict(lambda: {'conference': 0, 'championship': 0})
for participant_name, event_type, total_payout in cursor.fetchall():
    normalized_name = participant_name.strip().lower()
    if event_type.strip().lower() == 'conference winner':
        payouts[normalized_name]['conference'] = float(total_payout)
    elif event_type.strip().lower() == 'championship':
        payouts[normalized_name]['championship'] = float(total_payout)

cursor.close()
conn.close()

# Matchups and probabilities (example values for proof of concept)
team_probabilities = {
    "kansas city chiefs": {"semifinal_prob": 0.54, "championship_probs": {"washington commanders": 0.67, "philadelphia eagles": 0.51}},
    "buffalo bills": {"semifinal_prob": 0.46, "championship_probs": {"washington commanders": 0.68, "philadelphia eagles": 0.52}},
    "washington commanders": {"semifinal_prob": 0.28, "championship_probs": {"kansas city chiefs": 0.33, "buffalo bills": 0.32}},
    "philadelphia eagles": {"semifinal_prob": 0.72, "championship_probs": {"kansas city chiefs": 0.49, "buffalo bills": 0.48}},
}

# Function to calculate Portfolio EV
def calculate_portfolio_ev(winning_team, other_conference_teams):
    """
    Calculates Portfolio EV if `winning_team` wins the semifinal.
    """
    # Conference Winner EV
    conference_winner_ev = payouts[winning_team]['conference']

    for team in other_conference_teams:
        conference_winner_ev += team_probabilities[team]['semifinal_prob'] * payouts[team]['conference']
    
    # Championship EV
    championship_ev = 0
    for opponent, win_prob in team_probabilities[winning_team]["championship_probs"].items():
        opponent_prob = team_probabilities[opponent]["semifinal_prob"]
        championship_ev += (
            win_prob * opponent_prob * payouts[winning_team]["championship"] +
            (1 - win_prob) * opponent_prob * payouts[opponent]["championship"]
        )
    
    return conference_winner_ev + championship_ev

# Function to calculate Portfolio EV assuming a specific team in the other conference has won
def calculate_conditional_portfolio_ev(winning_team, other_conference_teams, conditioned_team):
    """
    Calculates Portfolio EV if `winning_team` wins the semifinal, assuming `conditioned_team` already won in the other conference.
    """
    # Save original probabilities
    original_probs = {team: team_probabilities[team]['semifinal_prob'] for team in other_conference_teams}

    # Set conditioned probabilities: 1 for `conditioned_team`, 0 for others
    for team in other_conference_teams:
        team_probabilities[team]['semifinal_prob'] = 1 if team == conditioned_team else 0

    # Compute EV
    ev = calculate_portfolio_ev(winning_team, other_conference_teams)

    # Restore original probabilities
    for team, prob in original_probs.items():
        team_probabilities[team]['semifinal_prob'] = prob

    return ev

# Teams in the two conferences
conference_a = ["kansas city chiefs", "buffalo bills"]
conference_b = ["washington commanders", "philadelphia eagles"]

# Calculate portfolio EVs for each scenario
ev_results = {}
conditional_ev_results = defaultdict(dict)

for winner in conference_a + conference_b:
    other_conference = conference_b if winner in conference_a else conference_a

    # Original Portfolio EV
    ev_results[winner] = calculate_portfolio_ev(winner, other_conference)

    # Conditional Portfolio EVs (assuming each team in the other conference has won)
    for conditioned_team in other_conference:
        conditional_ev_results[winner][conditioned_team] = calculate_conditional_portfolio_ev(winner, other_conference, conditioned_team)

# Compute EV deltas
ev_deltas = {
    "AFC Semifinal": abs(ev_results["kansas city chiefs"] - ev_results["buffalo bills"]),
    "NFC Semifinal": abs(ev_results["washington commanders"] - ev_results["philadelphia eagles"]),
}

# Print results
print("==== Portfolio EVs ====")
print(f"Portfolio EV if Kansas City Chiefs wins semifinal: ${ev_results['kansas city chiefs']:,.2f}")
print(f"Portfolio EV if Buffalo Bills wins semifinal: ${ev_results['buffalo bills']:,.2f}")
print(f"EV Delta: ${ev_deltas['AFC Semifinal']:,.2f}\n")

print(f"Portfolio EV if Washington Commanders wins semifinal: ${ev_results['washington commanders']:,.2f}")
print(f"Portfolio EV if Philadelphia Eagles wins semifinal: ${ev_results['philadelphia eagles']:,.2f}")
print(f"EV Delta: ${ev_deltas['NFC Semifinal']:,.2f}\n")

print("==== Conditional Portfolio EVs ====")
for conditioned_team, teams in {
    "kansas city chiefs": conference_b,
    "buffalo bills": conference_b,
    "washington commanders": conference_a,
    "philadelphia eagles": conference_a
}.items():
    print(f"\n--- If {conditioned_team.title()} already won ---")
    for team in teams:
        print(f"If {team.title()} wins semifinal: ${conditional_ev_results[team][conditioned_team]:,.2f}")


==== Portfolio EVs ====
Portfolio EV if Kansas City Chiefs wins semifinal: $244,026.39
Portfolio EV if Buffalo Bills wins semifinal: $47,758.46
EV Delta: $196,267.92

Portfolio EV if Washington Commanders wins semifinal: $127,342.75
Portfolio EV if Philadelphia Eagles wins semifinal: $164,009.96
EV Delta: $36,667.22

==== Conditional Portfolio EVs ====

--- If Kansas City Chiefs already won ---
If Washington Commanders wins semifinal: $235,819.90
If Philadelphia Eagles wins semifinal: $247,217.80

--- If Buffalo Bills already won ---
If Washington Commanders wins semifinal: $0.00
If Philadelphia Eagles wins semifinal: $66,331.20

--- If Washington Commanders already won ---
If Kansas City Chiefs wins semifinal: $235,819.90
If Buffalo Bills wins semifinal: $0.00

--- If Philadelphia Eagles already won ---
If Kansas City Chiefs wins semifinal: $247,217.80
If Buffalo Bills wins semifinal: $66,331.20


In [17]:
# missing EV delta in the output

import mysql.connector
from collections import defaultdict

# Connect to the database
conn = mysql.connector.connect(
    host='betting-db.cp86ssaw6cm7.us-east-1.rds.amazonaws.com',
    user='admin',
    password='7nRB1i2&A-K>',
    database='betting_db'
)

cursor = conn.cursor()

# Query to get PotentialPayouts for conference and championship bets
query = """
    SELECT 
        legs.ParticipantName,
        legs.EventType,
        SUM(bets.PotentialPayout) AS total_payout
    FROM bets
    JOIN legs ON bets.WagerID = legs.WagerID
    WHERE 
        bets.LegCount = 1
        AND bets.WLCA = 'Active'
        AND bets.WhichBankroll = 'GreenAleph'
        AND legs.LeagueName = 'NFL'
        AND legs.EventType IN ('Conference Winner', 'Championship')
    GROUP BY legs.ParticipantName, legs.EventType;
"""

cursor.execute(query)

# Store payouts in a dictionary
payouts = defaultdict(lambda: {'conference': 0, 'championship': 0})
for participant_name, event_type, total_payout in cursor.fetchall():
    normalized_name = participant_name.strip().lower()
    if event_type.strip().lower() == 'conference winner':
        payouts[normalized_name]['conference'] = float(total_payout)
    elif event_type.strip().lower() == 'championship':
        payouts[normalized_name]['championship'] = float(total_payout)

cursor.close()
conn.close()

# Matchups and probabilities (example values for proof of concept)
team_probabilities = {
    "kansas city chiefs": {"semifinal_prob": 0.54, "championship_probs": {"washington commanders": 0.67, "philadelphia eagles": 0.51}},
    "buffalo bills": {"semifinal_prob": 0.46, "championship_probs": {"washington commanders": 0.68, "philadelphia eagles": 0.52}},
    "washington commanders": {"semifinal_prob": 0.28, "championship_probs": {"kansas city chiefs": 0.33, "buffalo bills": 0.32}},
    "philadelphia eagles": {"semifinal_prob": 0.72, "championship_probs": {"kansas city chiefs": 0.49, "buffalo bills": 0.48}},
}

# Function to calculate Portfolio EV
def calculate_portfolio_ev(winning_team, other_conference_teams):
    """
    Calculates Portfolio EV if `winning_team` wins the semifinal.
    """
    # Conference Winner EV
    conference_winner_ev = payouts[winning_team]['conference']

    for team in other_conference_teams:
        conference_winner_ev += team_probabilities[team]['semifinal_prob'] * payouts[team]['conference']
    
    # Championship EV
    championship_ev = 0
    for opponent, win_prob in team_probabilities[winning_team]["championship_probs"].items():
        opponent_prob = team_probabilities[opponent]["semifinal_prob"]
        championship_ev += (
            win_prob * opponent_prob * payouts[winning_team]["championship"] +
            (1 - win_prob) * opponent_prob * payouts[opponent]["championship"]
        )
    
    return conference_winner_ev + championship_ev

# Function to calculate Portfolio EV assuming a specific team in the other conference has won
def calculate_conditional_portfolio_ev(winning_team, other_conference_teams, conditioned_team):
    """
    Calculates Portfolio EV if `winning_team` wins the semifinal, assuming `conditioned_team` already won in the other conference.
    """
    # Save original probabilities
    original_probs = {team: team_probabilities[team]['semifinal_prob'] for team in other_conference_teams}

    # Set conditioned probabilities: 1 for `conditioned_team`, 0 for others
    for team in other_conference_teams:
        team_probabilities[team]['semifinal_prob'] = 1 if team == conditioned_team else 0

    # Compute EV
    ev = calculate_portfolio_ev(winning_team, other_conference_teams)

    # Restore original probabilities
    for team, prob in original_probs.items():
        team_probabilities[team]['semifinal_prob'] = prob

    return ev

# Teams in the two conferences
conference_a = ["kansas city chiefs", "buffalo bills"]
conference_b = ["washington commanders", "philadelphia eagles"]

# Calculate portfolio EVs for each scenario
ev_results = {}
conditional_ev_results = defaultdict(dict)

for winner in conference_a + conference_b:
    other_conference = conference_b if winner in conference_a else conference_a

    # Original Portfolio EV
    ev_results[f"{winner.title()} wins semifinal"] = calculate_portfolio_ev(winner, other_conference)

    # Conditional Portfolio EVs (assuming each team in the other conference has won)
    for conditioned_team in other_conference:
        conditional_ev_results[winner.title()][f"If {conditioned_team.title()} wins semifinal"] = calculate_conditional_portfolio_ev(winner, other_conference, conditioned_team)

# Print results
print("==== Portfolio EVs ====")
for outcome, ev in ev_results.items():
    print(f"{outcome}: ${ev:,.2f}")

print("\n==== Conditional Portfolio EVs ====")
for winner, conditions in conditional_ev_results.items():
    print(f"\n--- if {winner} already won ---")
    for condition, ev in conditions.items():
        print(f"{condition}: ${ev:,.2f}")


==== Portfolio EVs ====
Kansas City Chiefs wins semifinal: $244,026.39
Buffalo Bills wins semifinal: $47,758.46
Washington Commanders wins semifinal: $127,342.75
Philadelphia Eagles wins semifinal: $164,009.96

==== Conditional Portfolio EVs ====

--- if Kansas City Chiefs already won ---
If Washington Commanders wins semifinal: $235,819.90
If Philadelphia Eagles wins semifinal: $247,217.80

--- if Buffalo Bills already won ---
If Washington Commanders wins semifinal: $0.00
If Philadelphia Eagles wins semifinal: $66,331.20

--- if Washington Commanders already won ---
If Kansas City Chiefs wins semifinal: $235,819.90
If Buffalo Bills wins semifinal: $0.00

--- if Philadelphia Eagles already won ---
If Kansas City Chiefs wins semifinal: $247,217.80
If Buffalo Bills wins semifinal: $66,331.20


In [3]:
# Missing conditional EV deltas

import mysql.connector
from collections import defaultdict

# Connect to the database
conn = mysql.connector.connect(
    host='betting-db.cp86ssaw6cm7.us-east-1.rds.amazonaws.com',
    user='admin',
    password='7nRB1i2&A-K>',
    database='betting_db'
)

cursor = conn.cursor()

# Query to get PotentialPayouts for conference and championship bets
query = """
    SELECT 
        legs.ParticipantName,
        legs.EventType,
        SUM(bets.PotentialPayout) AS total_payout
    FROM bets
    JOIN legs ON bets.WagerID = legs.WagerID
    WHERE 
        bets.LegCount = 1
        AND bets.WLCA = 'Active'
        AND bets.WhichBankroll = 'GreenAleph'
        AND legs.LeagueName = 'NFL'
        AND legs.EventType IN ('Conference Winner', 'Championship')
    GROUP BY legs.ParticipantName, legs.EventType;
"""

cursor.execute(query)

# Store payouts in a dictionary
payouts = defaultdict(lambda: {'conference': 0, 'championship': 0})
for participant_name, event_type, total_payout in cursor.fetchall():
    normalized_name = participant_name.strip().lower()
    if event_type.strip().lower() == 'conference winner':
        payouts[normalized_name]['conference'] = float(total_payout)
    elif event_type.strip().lower() == 'championship':
        payouts[normalized_name]['championship'] = float(total_payout)

cursor.close()
conn.close()

# Matchups and probabilities (example values for proof of concept)
team_probabilities = {
    "kansas city chiefs": {"semifinal_prob": 0.54, "championship_probs": {"washington commanders": 0.67, "philadelphia eagles": 0.51}},
    "buffalo bills": {"semifinal_prob": 0.46, "championship_probs": {"washington commanders": 0.68, "philadelphia eagles": 0.52}},
    "washington commanders": {"semifinal_prob": 0.28, "championship_probs": {"kansas city chiefs": 0.33, "buffalo bills": 0.32}},
    "philadelphia eagles": {"semifinal_prob": 0.72, "championship_probs": {"kansas city chiefs": 0.49, "buffalo bills": 0.48}},
}

# Function to calculate Portfolio EV
def calculate_portfolio_ev(winning_team, same_conference_teams, other_conference_teams):
    # Conference Winner EV
    # 100% win for this team
    conference_winner_ev = payouts[winning_team]['conference']  # 100% win for this team

    # for Team X in other conference:
    # += (Probability of Team X Winning Conferene) * (Conference Winner Potential Payout for Team X)
    for team in other_conference_teams:
        conference_winner_ev += (
            team_probabilities[team]['semifinal_prob'] *
            payouts[team]['conference']
        )
    
    # Championship EV
    # 100% probability this team reaches championship
    # for Team X in other conference:
    # += (Prob of this team beating Team X in Champ.) * (Champ. Payout for this team) + (Prob. of Team X beating this team in Champ.) * (Champ. Payout for Team X)
    championship_ev = 0
    for opponent, win_prob in team_probabilities[winning_team]["championship_probs"].items():
        opponent_prob = team_probabilities[opponent]["semifinal_prob"]
        championship_ev += (
            win_prob * opponent_prob * payouts[winning_team]["championship"] +
            (1 - win_prob) * opponent_prob * payouts[opponent]["championship"]
        )
    
    # Total Portfolio EV
    return conference_winner_ev + championship_ev

# Teams in the two conferences
conference_a = ["kansas city chiefs", "buffalo bills"]
conference_b = ["washington commanders", "philadelphia eagles"]

# Calculate portfolio EVs for each scenario
scenarios = [
    ("kansas city chiefs", conference_a, conference_b),
    ("buffalo bills", conference_a, conference_b),
    ("washington commanders", conference_b, conference_a),
    ("philadelphia eagles", conference_b, conference_a),
]

ev_results = {}
for winner, same_conference, other_conference in scenarios:
    ev = calculate_portfolio_ev(winner, same_conference, other_conference)
    ev_results[f"{winner.title()} wins semifinal"] = ev

# print EV results
for outcome, ev in ev_results.items():
    print(f"Portfolio EV if {outcome}: ${ev:.2f}")


Portfolio EV if Kansas City Chiefs wins semifinal: $244026.39
Portfolio EV if Buffalo Bills wins semifinal: $47758.46
Portfolio EV if Washington Commanders wins semifinal: $127342.75
Portfolio EV if Philadelphia Eagles wins semifinal: $164009.96
